In [ ]:
from smtutil import hiveutil, oracle_to_hive
import pandas as pd
from pyspark.sql.functions import lag, lead, first, last, udf, lit, col, array_sort, collect_list, expr, row_number, lit,concat
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, DateType, TimestampType, StructType, StructField, DoubleType, BooleanType
from pyspark.sql import functions as func
from datetime import datetime as dt
import datetime
import json
from datetime import date, timedelta, datetime
import os
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)
from typing import Union

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [2]:
# spark.stop()
# conn.close()
# sc.stop()

In [3]:
sc, spark = hiveutil.get_sc_spark()

In [4]:
from weblog_util.common import getCleansedDomainList, getAggValue, getDomainSummary

In [5]:
conn=oracle_to_hive.connect_to_Oracle()

Connect to Oracle Successfully


## Raw Data

In [6]:
spark.read.parquet("/app/ws-ic/hive/prd_biz_summ_vw.db/vw_shk_postpaid_master_summ/")\
.createOrReplaceTempView("vw_shk_postpaid_master_summ")
# spark.read.parquet("/app/ws-ic/hive/weblog.db/t_domain_dictionary/").createOrReplaceTempView('t_domain_dictionary')
spark.read.option("header",True).csv("/app/ws-sunny/git_repo/smt-churn-driver-analysis/data/raw/plan group definition 2021_11_17 v3.csv")\
.createOrReplaceTempView('plan_group')

In [7]:
spark.table("vw_shk_postpaid_master_summ").limit(5).toPandas()

TRX_MONTH  CUST_NUM      SUBR_NUM HKID_FLAG GENDER   AGE PREPOST_FLAG  \
0  2021-07-01  06632439      62084922         I      F    64           PO   
1  2021-07-01  12416302      97457796         I      F    39           PO   
2  2021-07-01  00535819      93868810         I      M    43           PO   
3  2021-07-01  10566247  450000107844         B         -999           PO   
4  2021-07-01  10566247  450000104697         B         -999           PO   

  REVENUE_SUBR_FLAG ACTIVE_SUBR_FLAG CRM_LABEL SUBR_SW_ON_DATE TENURE  \
0                 Y                Y       BE3      2013-11-22   2808   
1                 Y                Y       BE0      2021-04-28     94   
2                 Y                Y       BE4      1997-06-17   8810   
3                 Y                Y       AAV      2017-07-03   1489   
4                 Y                Y       AAV      2017-01-06   1667   

  RATE_PLAN_CARD_TYPE RATE_PLAN_TARIFF              CURRENT_RATE_PLAN_GROUP  \
0               5GAAL            138.0                        5G Add-on SIM   
1               5GAAC            458.0  5G Smartphone plan (SIM only) - Mid   
2               5GAAC            458.0  5G Smartphone plan (SIM only) - Mid   
3               W4TES              0.0             Data SIM Only Plan (MBB)   
4               W4TES              0.0             Data SIM Only Plan (MBB)   

  CURRENT_RATE_PLAN_OFFER_TYPE  \
0                     SIM_ONLY   
1                     SIM_ONLY   
2                     SIM_ONLY   
3                     SIM_ONLY   
4                     SIM_ONLY   

                             CURRENT_SHK_PLAN_GRP CURRENT_SHK_PLAN_GRP_ORDER  \
0                                 5G Mobile plans                        002   
1                                 5G Mobile plans                        001   
2                                 5G Mobile plans                        001   
3  Other plans (e.g. other voice plan, MBB plans)                        050   
4  Other plans (e.g. other voice plan, MBB plans)                        050   

                                             CURRENT_SHK_PLAN_SUBGRP  \
0                                                      5G Add-on SIM   
1     Mid tier (>50GB to <200GB) with add-on SIM without add-on pack   
2  Mid tier (>50GB to <200GB) without add-on SIM without add-on pack   
3                     Other plans (e.g. other voice plan, MBB plans)   
4                     Other plans (e.g. other voice plan, MBB plans)   

  CURRENT_SHK_PLAN_SUBGRP_ORDER PREVIOUS_CHANGE_PLAN_DATE  \
0                           040                2021-07-31   
1                           022                1900-01-01   
2                           024                2020-09-26   
3                           000                1900-01-01   
4                           000                1900-01-01   

  PREVIOUS_RATE_PLAN_CODE                 PREVIOUS_RATE_PLAN_GROUP  \
0                    3GNO  Smartphone voice & data sharing (SIM)-M   
1                                                                    
2                   3GBEW        Smartphone plan (SIM only) - High   
3                                                                    
4                                                                    

  PREVIOUS_RATE_PLAN_OFFER_TYPE        PREVIOUS_SHK_PLAN_GRP  \
0                      SIM_ONLY  SuperCare Mobile Mass plans   
1                                                              
2                      SIM_ONLY  SuperCare Mobile Mass plans   
3                                                              
4                                                              

  PREVIOUS_SHK_PLAN_GRP_ORDER  \
0                         010   
1                               
2                         010   
3                               
4                               

                          PREVIOUS_SHK_PLAN_SUBGRP  \
0      Family plans (Main SIM) without add-on pack   
1                           

## Customer Base

In [8]:
month_id = '202211'

In [9]:
# cust base
# sdf_cust_base = spark.sql(f"""
# select 
#   cust_num, 
#   subr_num, 
#   date_format(TRX_MONTH, 'yyyy-MM') as asof_month, 
#   a.CURRENT_SHK_PLAN_SUBGRP, 
#   b.Plan_group_lv4_Main_vs_Add_on, 
#   date_format(
#     LATEST_LD_EXP_DATE_SIM_OR_HSO, 'yyyy-MM'
#   ) as ce_month 
# from 
#   vw_shk_postpaid_master_summ a 
#   left join plan_group b on a.CURRENT_SHK_PLAN_SUBGRP = b.CURRENT_SHK_PLAN_SUBGRP 
# where 
#   NOMINATION_FLG = 'N' 
#   and a.CURRENT_SHK_PLAN_SUBGRP not in ('Excluded', 'MVNOs (Local)') 
#   and PREPOST_FLAG = 'PO' 
#   and REVENUE_SUBR_FLAG = 'Y' 
#   and ACTIVE_SUBR_FLAG = 'Y' 
#   and date_format(TRX_MONTH, 'yyyy-MM')= "{asof_month}"
# """)

In [35]:
sdf_cust_base = spark.sql(f"""
select 
  CUST_NUM, 
  SUBR_NUM, 
  date_format(TRX_MONTH, 'yyyyMM') as TRX_MONTH, 
  a.CURRENT_SHK_PLAN_SUBGRP, 
  b.Plan_group_lv4_Main_vs_Add_on, 
  date_format(
    LATEST_LD_EXP_DATE_SIM_OR_HSO, 'yyyyMM'
  ) as ce_month 
from 
  vw_shk_postpaid_master_summ a 
  left join plan_group b on a.CURRENT_SHK_PLAN_SUBGRP = b.CURRENT_SHK_PLAN_SUBGRP 
where 
  NOMINATION_FLG = 'N' 
  and a.CURRENT_SHK_PLAN_SUBGRP not in ('Excluded', 'MVNOs (Local)') 
  and PREPOST_FLAG = 'PO' 
  and REVENUE_SUBR_FLAG = 'Y' 
  and ACTIVE_SUBR_FLAG = 'Y' 
  and date_format(TRX_MONTH, 'yyyyMM')= "{month_id}"
""")

In [36]:
# hkid
sdf_hkid = spark.read.parquet('/app/ws-ic/hive/smc_lastest_image.db/hkid_br/')

In [37]:
# join customer base with hkid
sdf = sdf_cust_base.join(sdf_hkid, 'CUST_NUM', 'left').cache()

In [38]:
sdf.limit(5).toPandas()

CUST_NUM  SUBR_NUM TRX_MONTH  \
0  00004071  90181217    202211   
1  00009402  90126309    202211   
2  00034425  90110719    202211   
3  00038454  90114591    202211   
4  00039894  90113986    202211   

                                           CURRENT_SHK_PLAN_SUBGRP  \
0                                Speed capped unlimited data plans   
1                                                  ExtraCare plans   
2  Low tier (50GB or below) without add-on SIM without add-on pack   
3                  High tier (5GB or above) plans with add-on pack   
4                                Speed capped unlimited data plans   

  Plan_group_lv4_Main_vs_Add_on ce_month     HKID_BR  
0          SpeedCapped - Single   202405  D058500(A)  
1          SpeedCapped - Single   202311  E231916(4)  
2       SuperCare - 5G - Single   202409  A908839(6)  
3     SuperCare - 4.5G - Single   202303  E541978(A)  
4          SpeedCapped - Single   202308  D535418(9)

In [39]:
sdf_9mth_up = sdf.filter(F.col('ce_month')>=202308).cache() # 1 and 0 then groupby sum
print(sdf_9mth_up.count())

684223


In [40]:
sdf_9mth_up.limit(4).toPandas()

CUST_NUM  SUBR_NUM TRX_MONTH  \
0  00004071  90181217    202211   
1  00009402  90126309    202211   
2  00034425  90110719    202211   
3  00039894  90113986    202211   

                                           CURRENT_SHK_PLAN_SUBGRP  \
0                                Speed capped unlimited data plans   
1                                                  ExtraCare plans   
2  Low tier (50GB or below) without add-on SIM without add-on pack   
3                                Speed capped unlimited data plans   

  Plan_group_lv4_Main_vs_Add_on ce_month     HKID_BR  
0          SpeedCapped - Single   202405  D058500(A)  
1          SpeedCapped - Single   202311  E231916(4)  
2       SuperCare - 5G - Single   202409  A908839(6)  
3          SpeedCapped - Single   202308  D535418(9)

In [41]:
# sdf_9mth_up_count = sdf_9mth_up.groupby('HKID_BR').agg(F.count("SUBR_NUM").alias("NO_OF_LINE_WITH_CE_9MTH_UP"))

In [42]:
# join with count
# sdf_9mth_up = sdf_9mth_up.join(sdf_9mth_up_count, 'HKID_BR', 'left').cache()
# sdf_9mth_up.limit(4).toPandas()

In [44]:
sdf = sdf_9mth_up.drop_duplicates(['HKID_BR'])
print(sdf.count())
sdf = sdf.drop_duplicates(['SUBR_NUM'])
print(sdf.count())

510908
510908


## End of Customer Base

## Filter Table

In [20]:
# # source_table = pd.read_csv('./game_and_music_and_football_lover_with_membership_gender.csv')
# source_table = pd.read_csv('./game_and_music_and_football_lover_with_membership_gender.csv', converters = {'SUBR_NUM': str})

# # PREPROCESSING
# source_table = source_table[source_table['SEGMENT_TYPE']=='Narrowband']
# source_table['CUST_NUM'] = source_table['CUST_NUM'].apply(lambda x: '0' * (8 - len(str(x))) + str(x) if (8 - len(str(x))) > 0 else str(x))

# filter_table = spark.createDataFrame(source_table[['CUST_NUM', 'SUBR_NUM']])

In [46]:
# filter_table
sdf_filter_table = sdf.select('HKID_BR', 
                              'CUST_NUM', 
                              'SUBR_NUM',
                              'CURRENT_SHK_PLAN_SUBGRP',
                              'Plan_group_lv4_Main_vs_Add_on')
                              # 'NO_OF_LINE_WITH_CE_9MTH_UP')
                              # F.col('ce_month').alias('CE_MONTH'))

print(sdf_filter_table.count())
sdf_filter_table.limit(4).toPandas()

510908


HKID_BR  CUST_NUM  SUBR_NUM  \
0  G194784(1)  12600174  46269994   
1  Y854163(7)  12659482  46276667   
2  R930420(6)  12737885  46277979   
3  R068447(2)  12656129  46278118   

                                             CURRENT_SHK_PLAN_SUBGRP  \
0    Low tier (50GB or below) without add-on SIM without add-on pack   
1  Mid tier (>50GB to <200GB) without add-on SIM without add-on pack   
2                    High tier (5GB or above) plans with add-on pack   
3                    High tier (5GB or above) plans with add-on pack   

  Plan_group_lv4_Main_vs_Add_on  
0       SuperCare - 5G - Single  
1       SuperCare - 5G - Single  
2     SuperCare - 4.5G - Single  
3     SuperCare - 4.5G - Single

## Domain Lists

### top 100 doamin by cat - from Sunny C

In [22]:
df_top_domains = pd.read_csv('./top_100_domain_for_each_category.csv')
for col in df_top_domains.columns:
    try:
        df_top_domains[col] = df_top_domains[col].str.upper()
    except AttributeError:
        pass
df_top_domains.head()

tag_gp       tag                     domain  distinct_subr_num  r
0  NEWS/INFORMATION  POLITICS            WWW.POLITICO.EU             483341  1
1  NEWS/INFORMATION  POLITICS            MEDIA.SALON.COM             177235  2
2  NEWS/INFORMATION  POLITICS          WWW.DAILYWIRE.COM             133596  3
3  NEWS/INFORMATION  POLITICS            ASIASOCIETY.ORG             123513  4
4  NEWS/INFORMATION  POLITICS  WWW.REALCLEARPOLITICS.COM              30001  5

In [31]:
# l_raw = ['https://www.octopus.com.hk/']
l_top100 = df_top_domains[df_top_domains['tag'].isin(['KINDERGARTENS', 'KIDS'])]['domain']

In [32]:
print(len(l_top100))
l_top100[:10]

83


1755         READER.HKEDCITY.NET
1756         WAPPS1.HKEDCITY.NET
1757         E.STAR.HKEDCITY.NET
1758        STATIC1.HKEDCITY.NET
1759       STATICM1.HKEDCITY.NET
1760       WWW.BRITISHCOUNCIL.HK
1761          APPS2.HKEDCITY.NET
1762         SEARCH.HKEDCITY.NET
1763    VIDEOPLAYER.HKEDCITY.NET
1764        CD1.EDB.HKEDCITY.NET
Name: domain, dtype: object

### Pri & Sec School (0.6k)

##### Matching School Level

In [29]:
school_list = pd.read_excel('/app/ws-simon/projects/weblog_call_guard_sale_20221216/school_list.xlsx')
school_web = school_list[['SchoolWebSite','SchoolLevelEng']][(~school_list['SchoolWebSite'].isnull()) & (school_list['SchoolLevelEng'].isin(['PRIMARY','SECONDARY']))]

# SchoolWebSite formatting
school_web['SchoolWebSite'] = school_web['SchoolWebSite'].apply(lambda x: x.upper().replace('HTTP://','').replace('HTTPS://','').replace('WWW.','').split('/')[0])

# duplicated SchoolLevelEng set to both
school_web.loc[school_web.SchoolWebSite.isin([x for x in school_web.SchoolWebSite[school_web.SchoolWebSite.duplicated()]]),'SchoolLevelEng'] = 'BOTH'

# drop duplicated
df_school_web = school_web.drop_duplicates()

In [49]:
# to pyspark table
spark.createDataFrame(df_school_web).createOrReplaceTempView('t_school_web_level')

In [30]:
df_school_web.head(2)

SchoolWebSite SchoolLevelEng
0  SHATINPUBLICSCHOOL.EDU.HK           BOTH
2               CKCPS.EDU.HK        PRIMARY

In [34]:
l_school_web = df_school_web['SchoolWebSite'].tolist()
print(len(l_school_web))
l_school_web[:10]

1033


['SHATINPUBLICSCHOOL.EDU.HK',
 'CKCPS.EDU.HK',
 'TTSCH.EDU.HK',
 'GAIASCHOOL.EDU.HK',
 'ISLAND.EDU.HK',
 'YLPOKOK.EDU.HK',
 'WSCSS.EDU.HK',
 'FTESPS.EDU.HK',
 'IHMS.EDU.HK',
 'PLPB.EDU.HK']

## t_domain_dictionary

In [58]:
sdf_t_domain = spark.read.parquet('/app/ws-ic/hive/weblog.db/t_domain_dictionary/')
print(sdf_t_domain.count())
sdf_t_domain.limit(5).toPandas()

967868


tag_gp             tag                       domain agg_level  \
0  <Unclassified>  <Unclassified>               WWW.GOOGLE.COM    domain   
1  <Unclassified>  <Unclassified>          PLAY.GOOGLEAPIS.COM    domain   
2  <Unclassified>  <Unclassified>  GOOGLEADS.G.DOUBLECLICK.NET    domain   
3  <Unclassified>  <Unclassified>           WWW.GOOGLEAPIS.COM    domain   
4  <Unclassified>  <Unclassified>          APP-MEASUREMENT.COM    domain   

   agg_value  
0          1  
1          2  
2          3  
3          4  
4          6

### eClass

In [68]:
sdf_eclass = sdf_t_domain.filter(
    (F.col('domain').like('%ECLASS%')) 
    & (F.substring(F.col('domain'), -2, 2) == 'HK')
)

In [80]:
l_eclass = sdf_eclass.select('domain').toPandas()['domain'].tolist()
print(len(l_eclass))
l_eclass[:10]

619


['UNILINKS.ECLASS.COM.HK',
 'WWW.ECLASS.COM.HK',
 'KENTVILLE.ECLASS.HK',
 'VIDEOLIB.ECLASS.COM.HK',
 'ECLASSP.MUNSANG.EDU.HK',
 'ECLASS.PUIKIUCOLLEGE.EDU.HK',
 'ECLASSK.MUNSANG.EDU.HK',
 'ECLASS.YINGWA.EDU.HK',
 'ECLASS.EVANGEL.EDU.HK',
 'IHMK.ECLASS.HK']

### Education Resource Web
##### 1. Edu-Kingdom (教育王國）
##### 2. Ohpama (Oh! 爸媽)
##### 3. Schooland (升學天地)
##### 4. Parent.edu.hk (家長智Net)
##### 5. Edu.gov.hk (教育局)
##### 6. Chsc.hk (家校會)

In [74]:
sdf_erw = sdf_t_domain.filter(
    (F.col('domain').like('%EDU-KINGDOM%'))
    | (F.col('domain').like('%OHPAMA%'))
    | (F.col('domain').like('%SCHOOLAND%')) 
    | (F.col('domain').like('%PARENT.EDU%'))
    | (F.col('domain').like('%EDB.GOV.HK%')) 
    | (F.col('domain').like('%CHSC.HK%')) 
)

In [75]:
l_erw = sdf_erw.select('domain').toPandas()['domain'].tolist()
print(len(l_erw))
l_erw[:10]

84


['WWW.PARENT.EDU.HK',
 'CDN.EDU-KINGDOM.COM',
 'DWA-MAEMB47N.EDB.GOV.HK',
 'EDBSCCM.EDB.GOV.HK',
 'PSP2020.CHSC.HK',
 'QEFEPMS.EDB.GOV.HK',
 'MAP.SCHOOLANDCOLLEGELISTINGS.COM',
 'SCHOOLAND.HK',
 'WPHONE.EDU-KINGDOM.COM',
 'DWA-MAEMB46N.EDB.GOV.HK']

### Parent Utility Apps
##### 1. 蜜蜂試卷
##### 2. 小學學習字詞表
##### 3. Pleco

In [76]:
sdf_pua = sdf_t_domain.filter(
    (F.col('domain').like('%LIQUCN%'))
    | (F.col('domain').like('%EDBCHINESE%'))
    | (F.col('domain').like('%PLECOAPPS%')) 
)

In [77]:
l_pua = sdf_pua.select('domain').toPandas()['domain'].tolist()
print(len(l_pua))
l_pua[:10]

16


['PLECOAPPS.COM',
 'WWW.EDBCHINESE.HK',
 'LIQUCN-COM.CDN.AMPPROJECT.ORG',
 'IMAGES-LIQUCN-COM.CDN.AMPPROJECT.ORG',
 'SKIN-LIQUCN-COM.CDN.AMPPROJECT.ORG',
 'IMAGES.LIQUCN.COM',
 'COUNT.LIQUCN.COM',
 'SKIN.LIQUCN.COM',
 'LIQUCN.COM',
 'WWW.LIQUCN.COM']

### Entertainment
##### 1. Roblox
##### 2. MineCraft
##### 3. Scratch

In [78]:
sdf_entertain = sdf_t_domain.filter(
    (F.col('domain').like('%ROBLOX%'))
    | (F.col('domain').like('%MINECRAFT%'))
    | (F.col('domain').like('%SCRATCH.MIT.EDU%')) 
)

In [79]:
l_entertain = sdf_entertain.select('domain').toPandas()['domain'].tolist()
print(len(l_entertain))
l_entertain[:10]

240


['MINECRAFT-WALLET.COMMERCE.GAMESERVICES.XBOXLIVE.COM',
 'MINECRAFT.FANDOM.COM',
 'LOTRMINECRAFTMOD.FANDOM.COM',
 'MINECRAFTSTORYMODE.FANDOM.COM',
 'ROBLOX.FANDOM.COM',
 'MINECRAFTPROD.MAELSTROM.GAMESERVICES.XBOXLIVE.COM',
 'MINECRAFTLIST.ORG',
 'MINECRAFT.MAKECODE.COM',
 'MINECRAFTSKINSTEALER.COM',
 'MINECRAFT-IDENTITY.COMMERCE.GAMESERVICES.XBOXLIVE.COM']

## Get Summary

In [97]:
def getDomainSummary_v2(agg_value_table:object, agg_freq:str, agg_period_start:str, agg_period_end:str,app_name:str, filter_table=None):
    """
    Aggregate following Weblog Features with Web Domain Agg Value
        -  Sum of Data Usage in byte 
        -  Maximum of Total Hit
        -  Maximum of Day Used

    :param agg_value_table: Web Domain Agg Value Table
    :param agg_freq: Aggeration Frequncy ('daily','weekly','monthly')
    :param agg_period_start: Aggreate period start time in following format:
                                - daily: yyyymmdd
                                - weekly: yyyymmdd
                                - monthly: yyyymm
    :param agg_period_end: Aggreate period end time with same format of agg_period_start
    :param app_name: Column of Application name in the output 
    :param filter_table: Optional, inner join table on `SUBR_NUM` 

    :returns: Temp View of Aggerate Domain Summary:
            
    :raises ValueError: When You have input wrong agg_freq
    
    TODO:
    1. Add CUST_NUM from filter_table if it exists.
    """

    if int(agg_period_start) > int(agg_period_end):
            correct_agg_period_end=agg_period_start
            agg_period_start=agg_period_end
            agg_period_end=correct_agg_period_end

    if agg_freq == 'daily':
            t_domain_table='t_domain_daily'
            t_domain_time_key='date_id'
    elif agg_freq == 'weekly':
        t_domain_table='t_domain_weekly'
        t_domain_time_key='week_id'
    elif agg_freq == 'monthly':
        t_domain_table='t_domain_monthly'
        t_domain_time_key='month_id'
    else:
        raise ValueError("Wrong agg_freq received. Please input correct agg_freq (daily, weekly, monthly)")

    if agg_period_start == agg_period_end:
        sdf_domain_table=spark.read.parquet(f"/app/ws-ic/hive/weblog.db/{t_domain_table}/{t_domain_time_key}={agg_period_start}/")
        # add month_id column
        sdf_domain_table = sdf_domain_table.withColumn('month_id', F.lit(agg_period_start))
    else:
        sdf_domain_table=spark.read.parquet(f"/app/ws-ic/hive/weblog.db/{t_domain_table}").where(f"{t_domain_time_key} between {agg_period_start} and  {agg_period_end}")
    
    sdf_domain_table=sdf_domain_table.join(agg_value_table,sdf_domain_table.agg_value==agg_value_table.agg_value,'inner').select(sdf_domain_table['*'])
    
    for col in sdf_domain_table.columns:
        sdf_domain_table=sdf_domain_table.withColumnRenamed(col, col.upper())
    
    if filter_table:
        sdf_domain_table=sdf_domain_table.join(filter_table,sdf_domain_table.SUBR_NUM==filter_table.SUBR_NUM,'inner').select(sdf_domain_table['*'])

    groupby_list= ['SUBR_NUM',t_domain_time_key]
    if agg_freq == 'daily':
        res_domain=sdf_domain_table.groupby(groupby_list).agg(F.sum('BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),
                                                              F.sum('TOTAL_HIT').alias('TOTAL_HIT')
                                                             ).fillna(0).withColumn('APPS',F.lit(app_name))
    else:
        res_domain=sdf_domain_table.groupby(groupby_list).agg(F.sum('BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),
                                                              F.sum('TOTAL_HIT').alias('TOTAL_HIT'),
                                                              F.max('DAYS').alias('MAX_DAYS')
                                                             ).fillna(0).withColumn('APPS',F.lit(app_name))
    
    return res_domain

In [94]:
val_agg_period_start, val_agg_period_end = 202211, 202211
# t_domain_table='t_domain_monthly'
# t_domain_time_key='month_id'

In [95]:
# domain list , app list, result list
domain_lists = [l_top100, l_school_web, l_eclass, l_erw, l_pua, l_entertain]
app_name_list = ['KIDS_TOP100', 'SCHOOl_WEB', 'ECLASS', 'EDUCATION_RESOURCE_WEB', 'PARENT_UTILITY_APPS', 'ENTERTAINMENT']

summary_list = []

In [98]:
# domain list, app name list
for dl, a in zip(domain_lists, app_name_list):

    summary = getDomainSummary_v2(agg_value_table=getAggValue(dl),
                                  agg_freq='monthly',
                                  agg_period_start=val_agg_period_start,
                                  agg_period_end=val_agg_period_end,
                                  app_name=a,
                                  filter_table=sdf_filter_table).cache()
    
    print(summary.limit(5).toPandas())
    summary_list.append(summary)
    print(f'[Info] Finish summary *{a}*')

   SUBR_NUM  month_id  TOTAL_BYTE_COUNT  TOTAL_HIT  MAX_DAYS         APPS
0  46262646    202211          10942737         12         2  KIDS_TOP100
1  51006860    202211            957947         31        10  KIDS_TOP100
2  51100960    202211              9515          1         1  KIDS_TOP100
3  51101270    202211            150818         17         8  KIDS_TOP100
4  51113058    202211            190159         16         8  KIDS_TOP100
[Info] Finish summary *KIDS_TOP100*
   SUBR_NUM  month_id  TOTAL_BYTE_COUNT  TOTAL_HIT  MAX_DAYS        APPS
0  46262646    202211           3468003        445        27  SCHOOl_WEB
1  46278082    202211           1609213        198        26  SCHOOl_WEB
2  46410007    202211           1622574        213        25  SCHOOl_WEB
3  51006860    202211          10459460       1022        28  SCHOOl_WEB
4  51100960    202211            227505         26         3  SCHOOl_WEB
[Info] Finish summary *SCHOOl_WEB*
   SUBR_NUM  month_id  TOTAL_BYTE_COUNT  TOTAL_

## Pivot & Merge

In [205]:
# merge all summaries
res_merged = summary_list[0]

for summary in summary_list[1:]:
    res_merged = res_merged.union(summary).cache()

In [206]:
res_merged.limit(2).show()

+--------+--------+----------------+---------+--------+-----------+
|SUBR_NUM|month_id|TOTAL_BYTE_COUNT|TOTAL_HIT|MAX_DAYS|       APPS|
+--------+--------+----------------+---------+--------+-----------+
|46262646|  202211|        10942737|       12|       2|KIDS_TOP100|
|51006860|  202211|          957947|       31|      10|KIDS_TOP100|
+--------+--------+----------------+---------+--------+-----------+



In [207]:
res_merged.count()

1251428

In [208]:
# temp=[res_house_moving, res_house_renovating, res_hkbn, res_netvigator, res_village_property_agent]
# for x in temp:
#     print(x.count())

In [209]:
res_pivot=res_merged.groupBy('month_id','SUBR_NUM')\
                    .pivot('APPS').agg(F.sum('TOTAL_BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),\
                                      ).na.fill(0).withColumnRenamed('month_id','MONTH_ID')

In [210]:
res_pivot.limit(2).toPandas()

MONTH_ID  SUBR_NUM  ECLASS  EDUCATION_RESOURCE_WEB  ENTERTAINMENT  \
0    202211  92584886       0                  237853          21787   
1    202211  60908324       0                   21117           8852   

   KIDS_TOP100  PARENT UTILITY APPS  SCHOOl_WEB  
0       214797                    0     1966179  
1         8852                    0     2189662

#### Refine Customer Base

In [211]:
sdf.limit(1).toPandas()

CUST_NUM  SUBR_NUM TRX_MONTH  \
0  12600174  46269994    202211   

                                           CURRENT_SHK_PLAN_SUBGRP  \
0  Low tier (50GB or below) without add-on SIM without add-on pack   

  Plan_group_lv4_Main_vs_Add_on ce_month     HKID_BR  
0       SuperCare - 5G - Single   202310  G194784(1)

In [212]:
sdf_to_join = sdf.join(spark.table('plan_group').select('CURRENT_SHK_PLAN_GRP', 'CURRENT_SHK_PLAN_SUBGRP'), 'CURRENT_SHK_PLAN_SUBGRP', 'left').cache()

In [213]:
sdf_to_join = sdf_to_join.select(
    'CUST_NUM',
    'SUBR_NUM',
    'CURRENT_SHK_PLAN_GRP',
    'CURRENT_SHK_PLAN_SUBGRP',
     F.col('ce_month').alias('CE_MONTH'),
)

In [214]:
sdf_to_join.limit(2).toPandas()

CUST_NUM  SUBR_NUM CURRENT_SHK_PLAN_GRP  \
0  12600174  46269994      5G Mobile plans   
1  12659482  46276667      5G Mobile plans   

                                             CURRENT_SHK_PLAN_SUBGRP CE_MONTH  
0    Low tier (50GB or below) without add-on SIM without add-on pack   202310  
1  Mid tier (>50GB to <200GB) without add-on SIM without add-on pack   202401

### joining

In [215]:
sdf_output = sdf_to_join.join(res_pivot.drop('MONTH_ID'), 'SUBR_NUM', 'left').na.fill(0)

In [216]:
sdf_output.limit(2).toPandas()

SUBR_NUM  CUST_NUM CURRENT_SHK_PLAN_GRP  \
0  46259929  10342862      5G Mobile plans   
1  46262646  12583602      5G Mobile plans   

                                        CURRENT_SHK_PLAN_SUBGRP CE_MONTH  \
0  Low tier (50GB or below) without add-on SIM with add-on pack   202409   
1                                                 5G Add-on SIM   202310   

   ECLASS  EDUCATION_RESOURCE_WEB  ENTERTAINMENT  KIDS_TOP100  \
0       0                       0              0            0   
1       0                   19666         134395     10942737   

   PARENT UTILITY APPS  SCHOOl_WEB  
0                    0           0  
1                    0     3468003

In [218]:
sdf_output.count()

510908

In [222]:
sdf_output.describe().toPandas()

summary              SUBR_NUM            CUST_NUM  \
0   count                510908              510908   
1    mean   8.208604600265606E7    9610594.22855974   
2  stddev  1.6406182020787645E7  2980175.5236204695   
3     min              44009747            00000124   
4     max              98899959            12847404   

          CURRENT_SHK_PLAN_GRP               CURRENT_SHK_PLAN_SUBGRP  \
0                       510905                                510908   
1                         None                                  None   
2                         None                                  None   
3                5G Lite plans           "True" unlimited data plans   
4  SuperCare Mobile Mass plans  Upper low tier plans (500MB - 1.4GB)   

             CE_MONTH              ECLASS EDUCATION_RESOURCE_WEB  \
0              510908              510908                 510908   
1  202387.78471270757  26978.495425790945      741258.6784215553   
2   59.32788290039325  1151800.5062795621   1.2831552843438171E8   
3              202308                   0                      0   
4              203201           368595680            86039164084   

        ENTERTAINMENT           KIDS_TOP100 PARENT UTILITY APPS  \
0              510908                510908              510908   
1  247646.43009700376    1253073.9329487893  12266.125549022525   
2   5323584.831646381  3.6964599352582455E7   282747.1351654067   
3                   0                     0                   0   
4           935889947           17923408203           164426203   

          SCHOOl_WEB  
0             510908  
1  2340603.082709607  
2  4423182.452330964  
3                  0  
4         1017422390

In [244]:
sdf_output = sdf_output.withColumnRenamed('PARENT UTILITY APPS', 'PARENT_UTILITY_APPS')

### Columns Rename

In [245]:
sdf_output_final = sdf_output.withColumn('ECLASS_FLAG', F.when(F.col('ECLASS')>0, 1).otherwise(0)) \
                             .withColumn('EDUCATION_RESOURCE_WEB_FLAG', F.when(F.col('EDUCATION_RESOURCE_WEB')>0, 1).otherwise(0)) \
                             .withColumn('ENTERTAINMENT_FLAG', F.when(F.col('ENTERTAINMENT')>0, 1).otherwise(0)) \
                             .withColumn('KIDS_TOP100_DOMAINS_FLAG', F.when(F.col('KIDS_TOP100')>0, 1).otherwise(0)) \
                             .withColumn('PARENT_UTILITY_APPS_FLAG', F.when(F.col('PARENT_UTILITY_APPS')>0, 1).otherwise(0)) \
                             .withColumn('SCHOOl_WEB_FLAG', F.when(F.col('SCHOOl_WEB')>0, 1).otherwise(0)) \

In [246]:
sdf_output_final = sdf_output_final.select(
    'SUBR_NUM',
    'CUST_NUM',
    'CURRENT_SHK_PLAN_GRP',
    'CURRENT_SHK_PLAN_SUBGRP',
    'CE_MONTH',
    'ECLASS_FLAG',
    'EDUCATION_RESOURCE_WEB_FLAG',
    'ENTERTAINMENT_FLAG',
    'KIDS_TOP100_DOMAINS_FLAG',
    'PARENT_UTILITY_APPS_FLAG',
    'SCHOOl_WEB_FLAG',
)

### Output to csv

In [257]:
df_summary = sdf_output_final.toPandas()

In [258]:
df_summary.to_csv('./output/call_guard_kids_summary_20221220.csv', index=False)
# save to parquet
# summary.write.mode('append').parquet(f'/app/ws-simon/projects/football_lover_20221111/data/')

In [259]:
sdf_output_final.describe().toPandas()

summary              SUBR_NUM            CUST_NUM  \
0   count                510908              510908   
1    mean   8.208604600265606E7    9610594.22855974   
2  stddev  1.6406182020787645E7  2980175.5236204695   
3     min              44009747            00000124   
4     max              98899959            12847404   

          CURRENT_SHK_PLAN_GRP               CURRENT_SHK_PLAN_SUBGRP  \
0                       510905                                510908   
1                         None                                  None   
2                         None                                  None   
3                5G Lite plans           "True" unlimited data plans   
4  SuperCare Mobile Mass plans  Upper low tier plans (500MB - 1.4GB)   

             CE_MONTH           ECLASS_FLAG EDUCATION_RESOURCE_WEB_FLAG  \
0              510908                510908                      510908   
1  202387.78471270757  0.015636866128539775          0.3490393573794108   
2   59.32788290039325   0.12406604964181603         0.47666689532650747   
3              202308                     0                           0   
4              203201                     1                           1   

   ENTERTAINMENT_FLAG KIDS_TOP100_DOMAINS_FLAG PARENT_UTILITY_APPS_FLAG  \
0              510908                   510908                   510908   
1  0.5118279611984936       0.5833457295638353      0.07439695600773524   
2  0.4998605689448856      0.49300503556544856        0.262416050820905   
3                   0                        0                        0   
4                   1                        1                        1   

      SCHOOl_WEB_FLAG  
0              510908  
1  0.9151725946745793  
2  0.2786249604396545  
3                   0  
4                   1

In [272]:
pd.DataFrame(df_summary[['ECLASS_FLAG',
    'EDUCATION_RESOURCE_WEB_FLAG',
    'ENTERTAINMENT_FLAG',
    'KIDS_TOP100_DOMAINS_FLAG',
    'PARENT_UTILITY_APPS_FLAG',
    'SCHOOl_WEB_FLAG']].sum()).to_csv('./output/kids_sum.csv')

# Elderly

### Cust Base

In [150]:
sdf_cust_base_eld = spark.sql(f"""
select 
  CUST_NUM,
  SUBR_NUM,
  GENDER,
  AGE,
  a.CURRENT_SHK_PLAN_GRP,
  a.CURRENT_SHK_PLAN_SUBGRP, 
  date_format(
    LATEST_LD_EXP_DATE_SIM_OR_HSO, 'yyyyMM'
  ) as CE_MONTH
from 
  vw_shk_postpaid_master_summ a 
  left join plan_group b on a.CURRENT_SHK_PLAN_SUBGRP = b.CURRENT_SHK_PLAN_SUBGRP 
where 
  NOMINATION_FLG = 'N' 
  and a.CURRENT_SHK_PLAN_SUBGRP not in ('Excluded', 'MVNOs (Local)') 
  and PREPOST_FLAG = 'PO' 
  and REVENUE_SUBR_FLAG = 'Y' 
  and ACTIVE_SUBR_FLAG = 'Y' 
  and date_format(TRX_MONTH, 'yyyyMM')= "{month_id}"
""")

In [151]:
# hkid
sdf_hkid = spark.read.parquet('/app/ws-ic/hive/smc_lastest_image.db/hkid_br/')

In [186]:
# join customer base with hkid
sdf_cust_base_eld = sdf_cust_base_eld.join(sdf_hkid, 'CUST_NUM', 'left').cache()

In [153]:
sdf_cust_base_eld.limit(5).toPandas()

CUST_NUM  SUBR_NUM GENDER AGE               CURRENT_SHK_PLAN_GRP  \
0  00004071  90181217      M  70  Speed capped unlimited data plans   
1  00009402  90126309      M  74                    ExtraCare plans   
2  00034425  90110719      M  73                    5G Mobile plans   
3  00038454  90114591      M  69        SuperCare Mobile Mass plans   
4  00039894  90113986      M  55  Speed capped unlimited data plans   

                                           CURRENT_SHK_PLAN_SUBGRP CE_MONTH  \
0                                Speed capped unlimited data plans   202405   
1                                                  ExtraCare plans   202311   
2  Low tier (50GB or below) without add-on SIM without add-on pack   202409   
3                  High tier (5GB or above) plans with add-on pack   202303   
4                                Speed capped unlimited data plans   202308   

      HKID_BR  
0  D058500(A)  
1  E231916(4)  
2  A908839(6)  
3  E541978(A)  
4  D535418(9)

In [154]:
sdf_9mth_up_eld = sdf_cust_base_eld.filter(F.col('ce_month')>=202308).cache() # 1 and 0 then groupby sum
print(sdf_9mth_up_eld.count())

684223


In [155]:
sdf_9mth_up_eld.limit(4).toPandas()

CUST_NUM  SUBR_NUM GENDER AGE               CURRENT_SHK_PLAN_GRP  \
0  00004071  90181217      M  70  Speed capped unlimited data plans   
1  00009402  90126309      M  74                    ExtraCare plans   
2  00034425  90110719      M  73                    5G Mobile plans   
3  00039894  90113986      M  55  Speed capped unlimited data plans   

                                           CURRENT_SHK_PLAN_SUBGRP CE_MONTH  \
0                                Speed capped unlimited data plans   202405   
1                                                  ExtraCare plans   202311   
2  Low tier (50GB or below) without add-on SIM without add-on pack   202409   
3                                Speed capped unlimited data plans   202308   

      HKID_BR  
0  D058500(A)  
1  E231916(4)  
2  A908839(6)  
3  D535418(9)

### Activation Channel

In [170]:
print(datetime.now())
df_act_ch = pd.read_sql('''
    select 
        SUBR_NUM, 
        CHANNEL_EFFORT,
        row_number() over (partition by SUBR_NUM order by INV_DATE asc) RANK
    from
        PRD_BIZ_SUMM_VW.VW_RBD_ALL_CHANNEL_ACT_RE_TX_U
    ''', con=conn)
print(datetime.now())
# select SUBR_NUM, Channel_Effort PRD_BIZ_SUMM_VW.VW_RBD_ALL_CHANNEL_ACT_RE_TX_U
# (need min the INV_DATE)

2022-12-20 10:38:24.955246
2022-12-20 10:45:32.535431


In [172]:
df_act_ch.to_csv('./activate_channel.csv', index=False)

In [183]:
sdf_act_ch = spark.read.option("header",True).csv('./activate_channel.csv')
sdf_act_ch = sdf_act_ch.filter(F.col('RANK')==1).drop('RANK').cache()
sdf_act_ch.limit(5).toPandas()

SUBR_NUM CHANNEL_EFFORT
0  15000005       Outbound
1  15000010         Others
2  15000037            RBD
3  15000042            RBD
4  15000045            RBD

In [166]:
# windowSpec_eld  = Window.partitionBy("SUBR_NUM").orderBy(F.col("INV_DATE").asc())
# sdf_act_ch = sdf_act_ch.withColumn("ROW_NUMBER", F.row_number().over(windowSpec_eld))

In [184]:
# join with cust base eld
sdf_9mth_up_eld_with_act = sdf_9mth_up_eld.join(sdf_act_ch, 'SUBR_NUM', 'left')
print(sdf_9mth_up_eld_with_act.count())
sdf_9mth_up_eld_with_act.limit(5).toPandas()

684223


SUBR_NUM  CUST_NUM GENDER AGE         CURRENT_SHK_PLAN_GRP  \
0  46252242  12504208      F  62              5G Mobile plans   
1  46267377  07403563      M  50  SuperCare Mobile Mass plans   
2  46269994  12600174      M  56              5G Mobile plans   
3  46276667  12659482      M  19              5G Mobile plans   
4  46277979  12737885      F  35  SuperCare Mobile Mass plans   

                                             CURRENT_SHK_PLAN_SUBGRP CE_MONTH  \
0                                                      5G Add-on SIM   202310   
1                    SuperCare Add-on SIM (Secondary SIM of 5G plan)   202402   
2    Low tier (50GB or below) without add-on SIM without add-on pack   202310   
3  Mid tier (>50GB to <200GB) without add-on SIM without add-on pack   202401   
4                    High tier (5GB or above) plans with add-on pack   202406   

      HKID_BR CHANNEL_EFFORT  
0  D474125(1)         Others  
1  K009595(8)         Others  
2  G194784(1)         Others  
3  Y854163(7)         Others  
4  R930420(6)         Others

In [191]:
sdf_eld = sdf_9mth_up_eld_with_act.drop_duplicates(['HKID_BR'])
print(sdf.count())
sdf_eld = sdf_eld.drop_duplicates(['SUBR_NUM'])
print(sdf.count())

510908
510908


In [127]:
# get eld domains
l_eld = [
    'www.sundaykiss.com',
    'www.hellotoby.com',
    'technic.com.hk',
    'https://www.sunlight.hk/',
    'https://www.maidshk.com/',
    'https://www.harmonyhomehelper.com/',
    'https://www.helperplace.com/',
    'http://www.starcarehk.com',
    'https://familyhelperagency.com/',
    'https://www.helpermatch.hk/',
]

summary_eld = getDomainSummary_v2(agg_value_table=getAggValue(l_eld),
                              agg_freq='monthly',
                              agg_period_start=val_agg_period_start,
                              agg_period_end=val_agg_period_end,
                              app_name='ELDERLY',
                              filter_table=sdf_filter_table).cache()

In [190]:
res_pivot_eld=summary_eld.groupBy('SUBR_NUM')\
                    .pivot('APPS').agg(F.sum('TOTAL_BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),\
                                      ).na.fill(0)

In [192]:
sdf_eld_output = sdf_eld.join(res_pivot_eld, 'SUBR_NUM', 'left')
sdf_eld_output.limit(5).toPandas()

SUBR_NUM  CUST_NUM GENDER AGE         CURRENT_SHK_PLAN_GRP  \
0  46252242  12504208      F  62              5G Mobile plans   
1  46267377  07403563      M  50  SuperCare Mobile Mass plans   
2  46269994  12600174      M  56              5G Mobile plans   
3  46276667  12659482      M  19              5G Mobile plans   
4  46277979  12737885      F  35  SuperCare Mobile Mass plans   

                                             CURRENT_SHK_PLAN_SUBGRP CE_MONTH  \
0                                                      5G Add-on SIM   202310   
1                    SuperCare Add-on SIM (Secondary SIM of 5G plan)   202402   
2    Low tier (50GB or below) without add-on SIM without add-on pack   202310   
3  Mid tier (>50GB to <200GB) without add-on SIM without add-on pack   202401   
4                    High tier (5GB or above) plans with add-on pack   202406   

      HKID_BR CHANNEL_EFFORT ELDERLY  
0  D474125(1)         Others    None  
1  K009595(8)         Others    None  
2  G194784(1)         Others    None  
3  Y854163(7)         Others    None  
4  R930420(6)         Others    None

In [193]:
# fill na
sdf_eld_output = sdf_eld_output.na.fill(0)
sdf_eld_output.describe().toPandas()

summary             SUBR_NUM            CUST_NUM  GENDER  \
0   count               510908              510908  510908   
1    mean  8.206409322076382E7   9609269.902959045    None   
2  stddev  1.641286767591086E7  2980944.3457082645    None   
3     min             44017399            00000124           
4     max             98899959            12849235       U   

                  AGE         CURRENT_SHK_PLAN_GRP  \
0              510908                       510908   
1  36.280109139023075                         None   
2  104.82747957301304                         None   
3                 -10                5G Lite plans   
4                  98  SuperCare Mobile Mass plans   

       CURRENT_SHK_PLAN_SUBGRP            CE_MONTH                HKID_BR  \
0                       510908              510908                 510907   
1                         None  202387.75126245822  2.4174598674941825E14   
2                         None   59.33299484244378   9.479516348111712E15   
3  "True" unlimited data plans              202308               00002604   
4         Unlimited data plans              203201             Z991875(3)   

  CHANNEL_EFFORT             ELDERLY  
0         509167              510908  
1           None   7698.450186726377  
2           None  298425.50403374736  
3             BM                   0  
4      Telesales            76768757

In [249]:
# add flags cols
sdf_eld_output = sdf_eld_output.withColumn('FEMALE_FLAG', F.when(F.col('GENDER')=='F', 1).otherwise(0)) \
                               .withColumn('AGED_FLAG', F.when(F.col('AGE').between(35, 44), 1).otherwise(0)) \
                               .withColumn('RBD_FLAG', F.when(F.col('CHANNEL_EFFORT')=='RBD', 1).otherwise(0)) \
                               .withColumn('ELDERLY_WEBSITES_FLAG', F.when(F.col('ELDERLY')>0, 1).otherwise(0)) \
# Aged_flag | Female_flag| RBD_Flag

In [250]:
sdf_eld_output.describe().toPandas()

summary             SUBR_NUM            CUST_NUM  GENDER  \
0   count               510908              510908  510908   
1    mean  8.206409322076382E7   9609269.902959045    None   
2  stddev  1.641286767591086E7  2980944.3457082645    None   
3     min             44017399            00000124           
4     max             98899959            12849235       U   

                  AGE         CURRENT_SHK_PLAN_GRP  \
0              510908                       510908   
1  36.280109139023075                         None   
2  104.82747957301304                         None   
3                 -10                5G Lite plans   
4                  98  SuperCare Mobile Mass plans   

       CURRENT_SHK_PLAN_SUBGRP            CE_MONTH                HKID_BR  \
0                       510908              510908                 510907   
1                         None  202387.75126245822  2.4174598674941825E14   
2                         None   59.33299484244378   9.479516348111712E15   
3  "True" unlimited data plans              202308               00002604   
4         Unlimited data plans              203201             Z991875(3)   

  CHANNEL_EFFORT             ELDERLY            AGE_FLAG         FEMALE_FLAG  \
0         509167              510908              510908              510908   
1           None   7698.450186726377  0.2536562355649158  0.4939813038746702   
2           None  298425.50403374736  0.4351035741870573  0.4999642632747805   
3             BM                   0                   0                   0   
4      Telesales            76768757                   1                   1   

            AGED_FLAG            RBD_FLAG ELDERLY_WEBSITES_FLAG  
0              510908              510908                510908  
1  0.2536562355649158  0.3433064269888121  0.006077415111918389  
2  0.4351035741870573  0.4748131900499463   0.07772060190533799  
3                   0                   0                     0  
4                   1                   1                     1

In [226]:
# df_temp = sdf_eld_output.groupBy('AGE').count().orderBy(F.col('AGE').desc()).toPandas()
# df_temp[df_temp['AGE'].astype(int)>=0]

In [251]:
# final filter
sdf_eld_output_final = sdf_eld_output.filter(F.col('AGE')>0)

In [252]:
# final selection of cols
sdf_eld_output_final = sdf_eld_output_final.select(
    'SUBR_NUM',
    'CUST_NUM',
    'GENDER',
    'FEMALE_FLAG',
    'AGE',
    'AGED_FLAG',
    F.col('CHANNEL_EFFORT').alias('ACTIVATION_CHANNEL'),
    'RBD_FLAG',
    'CURRENT_SHK_PLAN_GRP',
    'CURRENT_SHK_PLAN_SUBGRP',
    'CE_MONTH',
    'ELDERLY_WEBSITES_FLAG',
)
sdf_eld_output_final.limit(5).toPandas()

SUBR_NUM  CUST_NUM GENDER  FEMALE_FLAG AGE  AGED_FLAG ACTIVATION_CHANNEL  \
0  46252242  12504208      F            1  62          0             Others   
1  46267377  07403563      M            0  50          0             Others   
2  46269994  12600174      M            0  56          0             Others   
3  46276667  12659482      M            0  19          0             Others   
4  46277979  12737885      F            1  35          1             Others   

   RBD_FLAG         CURRENT_SHK_PLAN_GRP  \
0         0              5G Mobile plans   
1         0  SuperCare Mobile Mass plans   
2         0              5G Mobile plans   
3         0              5G Mobile plans   
4         0  SuperCare Mobile Mass plans   

                                             CURRENT_SHK_PLAN_SUBGRP CE_MONTH  \
0                                                      5G Add-on SIM   202310   
1                    SuperCare Add-on SIM (Secondary SIM of 5G plan)   202402   
2    Low tier (50GB or below) without add-on SIM without add-on pack   202310   
3  Mid tier (>50GB to <200GB) without add-on SIM without add-on pack   202401   
4                    High tier (5GB or above) plans with add-on pack   202406   

   ELDERLY_WEBSITES_FLAG  
0                      0  
1                      0  
2                      0  
3                      0  
4                      0

In [253]:
sdf_eld_output_final.toPandas().to_csv('./output/call_guard_elderly_summary_20221220.csv', index=False)

## End

In [ ]:
# df = df_summary[df_summary['TOTAL_BYTE_COUNT'] >= ]

In [41]:
fb_list = list(map(str, df_summary['SUBR_NUM']))

In [42]:
fb_list[0]

'92292817'

In [48]:
df2 = source_table.copy()
df2['SEGMENT'] = ['football_lover' if str(r['SUBR_NUM']) in fb_list else 'gamer' if r['GAME_LOVER_TOTAL_DAY_CNT'] >= 27 else 'streaming_lover' if r['MUSIC_LOVER_TOTAL_DAY_CNT'] >= 18 else 'na' for i, r in df2.iterrows()]

In [49]:
df_output = df2[['CUST_NUM', 'SUBR_NUM', 'SEGMENT']]
df_output = df_output[df_output['SEGMENT'] != 'na']
df_output.to_csv('./football_lover.csv', index=False)

In [2]:
/0

TypeError: 'int' object is not callable